In [3]:
using Pkg
Pkg.activate("..")

using BenchmarkTools
using StaticArrays
using ROOTIO

Activating environment at `~/Dev/ROOTIO.jl/Project.toml`


In [4]:
ROOTIO.@io struct Foo
    a::Int32
    b::Int64
    c::Float32
    d::SVector{5, UInt8}
end

In [5]:
buf = IOBuffer(Vector{UInt8}(1:sizeof(Foo)))

IOBuffer(data=UInt8[...], readable=true, writable=false, seekable=true, append=false, size=21, maxsize=Inf, ptr=1, mark=-1)

In [6]:
foo = ROOTIO.unpack(buf, Foo)

Foo(16909060, 361984551142689548, 4.377526f-31, UInt8[0x11, 0x12, 0x13, 0x14, 0x15])

┌ Info: Recompiling stale cache file /Users/tamasgal/.julia/compiled/v1.2/ROOTIO/koyqt.ji for ROOTIO [3cd96dde-e98d-4713-81e9-a4a1b0235ce9]
└ @ Base loading.jl:1240


Any[:(SVector{4, UInt8}), :Int32, :Int32, :UInt32, :UInt32, :Int32, :Int32, :Int32, :UInt8, :Int32, :UInt32, :Int32, :(SVector{18, UInt8})]
SVector{4, UInt8}
Expr
Int32
Symbol
Int32
Symbol
UInt32
Symbol
UInt32
Symbol
Int32
Symbol
Int32
Symbol
Int32
Symbol
UInt8
Symbol
Int32
Symbol
UInt32
Symbol
Int32
Symbol
SVector{18, UInt8}
Expr
Any[:(read(io, sizeof(SVector{4, UInt8}))), :(ntoh(read(io, Int32))), :(ntoh(read(io, Int32))), :(ntoh(read(io, UInt32))), :(ntoh(read(io, UInt32))), :(ntoh(read(io, Int32))), :(ntoh(read(io, Int32))), :(ntoh(read(io, Int32))), :(ntoh(read(io, UInt8))), :(ntoh(read(io, Int32))), :(ntoh(read(io, UInt32))), :(ntoh(read(io, Int32))), :(read(io, sizeof(SVector{18, UInt8})))]

In [4]:
@ROOTIO.io struct Bar
    a::Int32
    b::Float32
    c::SVector{4, UInt8}
end

Any[:Int32, :Float32, :(SVector{4, UInt8})]
Int32
Symbol
Float32
Symbol
SVector{4, UInt8}
Expr
Any[:(ntoh(read(io, Int32))), :(ntoh(read(io, Float32))), :(read(io, sizeof(SVector{4, UInt8})))]

In [5]:
@ROOTIO.io struct Baz
    a::Int32
    b::Float32
    c::SVector{4, UInt16}
end

Any[:Int32, :Float32, :(SVector{4, UInt16})]
Int32
Symbol
Float32
Symbol
SVector{4, UInt16}
Expr


LoadError: No parser found for type SVector{4, UInt16}

In [5]:
@ROOTIO.io struct Foo
    a::Int32
    b::Int32
    c::Float32
    d::Int32
end

Any[:Int32, :Int32, :Float32, :Int32]
Int32
Symbol
Int32
Symbol
Float32
Symbol
Int32
Symbol
Any[:(ntoh(read(io, Int32))), :(ntoh(read(io, Int32))), :(ntoh(read(io, Float32))), :(ntoh(read(io, Int32)))]

In [6]:
Foo(1, 2, 3, 4)

Foo(1, 2, 3.0f0, 4)

In [7]:
data = rand(UInt8, 1000000000);

In [8]:
buf = IOBuffer(data);

In [14]:
read(buf, 4)

UndefVarError: UndefVarError: buf not defined

In [10]:
buf

IOBuffer(data=UInt8[...], readable=true, writable=false, seekable=true, append=false, size=1000000000, maxsize=Inf, ptr=5, mark=-1)

In [12]:
ROOTIO.unpack(IOBuffer(rand(UInt8, 1000)), Foo)

Foo(769902277, 877929864, -1.9601652f-14, 917802855)

In [13]:
ROOTIO.unpack(IOBuffer(rand(UInt8, 1000)), Bar)

Bar(-1787811434, -2.3025051f-10, UInt8[0x5b, 0x6e, 0x2e, 0x63])

In [14]:
@btime ROOTIO.unpack($buf, Foo)

  8.548 ns (0 allocations: 0 bytes)


Foo(228380471, -1642559088, -3.3711032f30, 2050440449)

In [16]:
@btime ROOTIO.unpack($buf, Bar)

  45.611 ns (2 allocations: 96 bytes)


Bar(1947119411, -1.2519076f10, UInt8[0x96, 0x62, 0xaf, 0x54])

In [15]:
@btime retrieve_big_endian_32_foo_hardcoded_using_ntoh($data)

UndefVarError: UndefVarError: retrieve_big_endian_32_foo_hardcoded_using_ntoh not defined

In [23]:
@btime IOBuffer(data)

  85.815 ns (1 allocation: 64 bytes)


IOBuffer(data=UInt8[...], readable=true, writable=false, seekable=true, append=false, size=16, maxsize=Inf, ptr=1, mark=-1)

In [7]:
data = Vector{UInt8}([0x00, 0x00, 0x00, 0x64, 0x00, 0x00, 0x00, 0x32, 0x00, 0x00, 0x00, 0x16, 0x00, 0x00, 0x00, 0x24]);

In [2]:
struct Foo
    a::Int32
    b::Int32
    c::Float32
    d::Int32
end

In [21]:
function retrieve_foo(data)
    buf = IOBuffer(data)
    Foo(read(buf, Int32), read(buf, Int32), read(buf, Float32), read(buf, Int32))
end

function retrieve_foo_via_ref(data)
    ref = Ref{Foo}()
    read!(IOBuffer(data), ref)
    return ref[]
end


function retrieve_big_endian_32_foo(data)
    ref = Ref{Foo}()
    @inbounds for idx in range(1; length=Int(length(data)/4), step=4)
        reverse!(data, idx, idx+3)
    end
    read!(IOBuffer(data), ref)
    return ref[]
end


function retrieve_big_endian_32_foo_hardcoded_using_reverse_and_ref(data)
    ref = Ref{Foo}()
    reverse!(data, 1, 4)
    reverse!(data, 5, 8)
    reverse!(data, 9, 12)
    read!(IOBuffer(data), ref)
    return ref[]
end


function retrieve_big_endian_32_foo_hardcoded_using_ntoh(data)
    buf = IOBuffer(data)
    Foo(ntoh(read(buf, Int32)), ntoh(read(buf, Int32)), ntoh(read(buf, Float32)), ntoh(read(buf, Int32)))
end

retrieve_big_endian_32_foo_hardcoded_using_ntoh (generic function with 1 method)

In [15]:
data = Vector{UInt8}([0x00, 0x00, 0x00, 0x64, 0x00, 0x00, 0x00, 0x32, 0x00, 0x00, 0x00, 0x16, 0x00, 0x00, 0x00, 0x24]);

In [19]:
@btime retrieve_foo(data)

UndefVarError: UndefVarError: retrieve_foo not defined

In [9]:
@btime retrieve_foo_via_ref($data)

  15.397 ns (2 allocations: 96 bytes)


Foo(1677721600, 838860800, 1.0339758f-25)

In [10]:
@btime retrieve_big_endian_32_foo($data)

  38.039 ns (2 allocations: 96 bytes)


Foo(100, 50, 3.1f-44)

In [11]:
@btime retrieve_big_endian_32_foo_hardcoded_using_reverse_and_ref($data)

  27.803 ns (2 allocations: 96 bytes)


Foo(100, 50, 3.1f-44)

In [12]:
@btime retrieve_big_endian_32_foo_hardcoded_using_ntoh($data)

  12.002 ns (1 allocation: 64 bytes)


Foo(100, 50, 3.1f-44)